# Create video of the images in the folder

### Remove old variables

In [1]:
# Remove old variables
#%reset

### Import libraries

In [2]:
import cv2
import os
import numpy as np
from skimage import exposure, measure
from tqdm import tqdm

### Set paths and read images

In [3]:
# Set paths and video output name
image_folder = '../data/Varrio_Scanner2_no_crop_results'
video_name = '../data/Varrio_Scanner2_DL_Video_no_crop.mp4'

# Station name for the video
station = "Varrio2: "

# Set video frame size (scale)
scale=1

# Set font size
font_size=3

# Number of pictures in second
fps=5

### Get image paths and set video settings

In [4]:
# Get image names
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sort image names
images = sorted(images)

# Set video frame size
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
height=int(scale*height)
width=int(scale*width)
dim=(width, height)

# Set video settings
video = cv2.VideoWriter(filename=video_name, fourcc=cv2.VideoWriter_fourcc(*'XVID'), fps=fps, frameSize=dim)

# Set text font
font = cv2.FONT_HERSHEY_SIMPLEX

### Remove isolated pixels

In [5]:
# Filter noise by the pixel area
def remove_isolated_pixels(thresh, min_size, max_size):
    # Perform a connected component analysis on the thresholded
    # image and remove components that have size less than thershold
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > min_size and numPixels < max_size:
            mask = cv2.add(mask, labelMask)
            
    # Return cleaned mask
    return mask

### Convert pics to video and save output

In [6]:
# Initialize roots history image
im_hist = np.zeros((height,width), np.uint8)

# Make video of images
for image in tqdm(images):
    
    # Read image
    im_gray = cv2.imread(os.path.join(image_folder, image), 0)
    
    # Resize image
    im_gray = cv2.resize(im_gray, dim, interpolation = cv2.INTER_AREA)
    
    # Make it black and white
    ret, im_bw_tmp = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    
    # Make areas fatter and then back to thinner (remove small holes)
    im_bw_tmp = cv2.dilate(im_bw_tmp, None, iterations=2)
    im_bw_tmp = cv2.erode(im_bw_tmp, None, iterations=2)
    
    # Remove isolated pixels
    im_bw_tmp = remove_isolated_pixels(im_bw_tmp, min_size=75, max_size=1000)
    
    # Add new roots to history
    im_hist = im_hist + im_bw_tmp
    
    # Copy info to im, which will be used in video writing and make it RGB
    im = im_hist.copy()
    im = cv2.cvtColor(im, cv2.COLOR_GRAY2RGB)
    
    # Get image date from the image name
    #date = image[20:30] # For Hyde Scanners
    date = image[14:24] # For Varrio Scanners
    
    # Add date to picture upper left corner
    cv2.putText(img=im, text=station+date, org=(font_size*10,font_size*30), fontFace=font, fontScale=font_size, 
                color=(255, 255, 255), thickness=font_size, lineType=cv2.LINE_AA)
    
    # Add pic to video
    video.write(im)

# Close windows
cv2.destroyAllWindows()

# Save output video
video.release()

100%|██████████| 184/184 [01:06<00:00,  2.76it/s]
